In [43]:
from pharynx_analysis import (pharynx_io as pio, image_processing as ip, experiment, plots)
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from importlib import reload
from skimage import measure, transform
from collections import namedtuple
import pdb
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import scale
import skimage

In [40]:
img_path = "/Users/sean/code/wormAnalysis/data/paired_ratio_minimal/2017_02_27-HD233_HD236/2017_02_27-HD233_HD236.tif"
strain_map_path = "/Users/sean/code/wormAnalysis/data/paired_ratio_minimal/2017_02_27-HD233_HD236/indexer.csv"
strains = pio.load_strain_map_from_disk(strain_map_path)

imaging_scheme = "TL/470/410/470/410"

In [ ]:
plt.plot(ex.raw_intensity_data.sel(wavelength='410', pair=0).T);

In [41]:
raw_image_data = pio.load_images(img_path, imaging_scheme, strains)
seg_stack = ip.segment_pharynxes(raw_image_data)

rot_fl, rot_seg = ip.center_and_rotate_pharynxes(raw_image_data, seg_stack)

In [49]:
skimage.io.imsave('/Users/sean/Desktop/seg.tif', rot_seg.sel(wavelength='410', pair=0).data.astype(np.uint16))

/Users/sean/.local/share/virtualenvs/python-zUhjj0fC/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: /Users/sean/Desktop/seg.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
